In [1]:
import os; os.chdir('../')
import sys; sys.path.append('.')

In [2]:
%config InlineBackend.figure_format = 'retina'
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%config InlineBackend.figure_format = 'retina'
from IPython.core.display import display, HTML, Markdown
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline

In [3]:
import json
from datetime import datetime, timezone

In [9]:
from mongoengine.connection import disconnect
from importlib import reload
import py.classes as classes;reload(classes)
import py.mongo as mongo;reload(mongo)
import experiment_analysis.mongo_utils as mu;reload(mu)
disconnect()
db = mongo.connect_to_mongo(db_name='evo_v2')

import logging
l = logging.getLogger("my_logger")
l.setLevel(logging.INFO)


<module 'py.classes' from '/Users/userid/git/evotrade/py/classes.py'>

<module 'py.mongo' from '/Users/userid/git/evotrade/py/mongo.py'>

<module 'experiment_analysis.mongo_utils' from '/Users/userid/git/evotrade/experiment_analysis/mongo_utils.py'>

In [9]:
#SR sell 10M
dfs10, spoints10 = mu.get_experiment_info('5d0e9b16d71f100b8a48f460')

In [11]:
# let's try to find a better model for a step 97

In [12]:
from scipy.stats import randint as sp_randint

In [26]:
exp_id, steps = '5d0e9b16d71f100b8a48f460', dfs10[:20].index
MODEL_SPACE={'MODEL_SPACE':
                 {"max_depth": range(8,12,2), 
#                   "max_features": [1, 3, 10, 20]”
#                           "min_samples_split": [2, 3, 10],
#                           "bootstrap": [True, False],
#                           "criterion": ["gini", "entropy"],
                          'n_estimators':range(8, 12,2)
                 }
            }
n_iter=40

In [39]:
ex = mongo.Experiment.objects_safe(pk=exp_id).first()

In [ ]:
    datasets = None

    ec = classes.ExperimentConfiguration()
    ec.load_from_experiment_step(exp_id, 1, fine_tuned=False)
    for step in steps:
        root_point = mongo.Point.objects_safe(experiment = ex, fine_tuned__in=[None, False], step = step).first()
        ec.adjust_parameters(**root_point.coordinates)
        ec._load_from_dict(MODEL_SPACE)
        if datasets == None:
            datasets = classes.SKModel.get_datasets_from_exp_config(ec)
        else:
            for d in datasets: d.update(update_source=False)
        model = classes.SKModel(ec)
        model.train(
    #         model_params = {"max_depth": 10, 'n_estimators':10},
            datasets=datasets,
            tags='TRAIN', 
            model_param_distributions={"max_depth": sp_randint(1, 20), "n_estimators": sp_randint(10, 200),} ,
            random_search_iterations=n_iter)

        result_test, result_list_test = model.evaluate(datasets, tags='TEST')
        result_val, result_list_val = model.evaluate(datasets, tags='VAL')

        point = classes.Point(
                    step = step, 
                    evaluation_on_test = result_test.mongo_object_with_deals,
                    evaluation_on_val = result_val.mongo_object,
                    detailed_evaluation_on_val = [r.mongo_object for r in result_list_val],
                    detailed_evaluation_on_test = [r.mongo_object for r in result_list_test],
                    coordinates = root_point.coordinates,
                    experiment = ex.id,
                    test_days=result_test.days,
                    test_mean=result_test.mean,
                    test_std=result_test.std,
                    test_deals_per_day=result_test.deals_per_day,
                    test_diff=result_test.diff,
                    test_min=result_test.min,
                    test_max=result_test.max,
                    test_total=result_test.total,
                    val_days=result_val.days,
                    val_mean=result_val.mean,
                    val_std=result_val.std,
                    val_deals_per_day=result_val.deals_per_day,
                    val_diff=result_val.diff,
                    val_min=result_val.min,
                    val_max=result_val.max,
                    val_total=result_val.total,
                    clf_params=getattr(model.clf, 'best_params_', None),
                    fine_tuned=True
                    )
        point.save_safe()

In [12]:
i = 0
for p in mongo.Point.objects_safe(experiment='5d0d4f4cd71f100b8a48f450', fine_tuned=True):
    i += 1
print(i)

11


In [47]:
p.step = 1

In [48]:
p.save_safe()

<Point: Point object>

In [51]:
p.step

1

In [63]:
type(dfs10.values)

numpy.ndarray